Ejemplo de Elegi vivir:
https://colab.research.google.com/drive/1csMQIG_ZX523Gta8W92cWb3WmJWio3Dq?usp=sharing

## Coding Exercise #0509

In [1]:
import nltk
from numpy.random import randint, seed
from sklearn.feature_extraction.text import CountVectorizer

### 1. n-Gram based autofill:

In [2]:
!pip install EbookLib


In [7]:
!pip install ebooklib
!pip install beautifulsoup4

In [3]:
from ebooklib import epub
from bs4 import BeautifulSoup

def extract_text_from_epub(file_path):
    book = epub.read_epub(file_path)
    text = ''
    for item in book.get_items():
        if item.get_type() == epub.EpubHtml:
            soup = BeautifulSoup(item.content, 'html.parser')
            text += soup.get_text()
    return text

# Uso:
text = extract_text_from_epub('fabulasDeEsopo.epub')
print(text[:1000])  # Muestra los primeros 1000 caracteres


In [2]:
from ebooklib import epub
from bs4 import BeautifulSoup
import ebooklib

def extraer_texto_epub(ruta):
    libro = epub.read_epub(ruta)
    texto_total = ''
    for item in libro.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            contenido = item.get_content()
            soup = BeautifulSoup(contenido, 'html.parser')
            texto_total += soup.get_text()
    return texto_total


In [3]:
text = extraer_texto_epub('fabulasDeEsopo.epub')

In [4]:
text

'\n\n\n\n01 - El águila, el cuervo y el pastor.\nLanzándose desde una cima, un águila arrebató a un corderito.\nLa vio un cuervo y tratando de imitar al águila, se lanzó sobre un carnero, pero con tan mal conocimiento en el arte que sus garras se enredaron en la lana, y batiendo al máximo sus alas no logró soltarse.\nViendo el pastor lo que sucedía, cogió al cuervo, y cortando las puntas de sus alas, se lo llevó a sus niños.\nLe preguntaron sus hijos acerca de que clase de ave era aquella, y él les dijo:\n—Para mí, sólo es un cuervo; pero él, se cree águila.\nPon tu esfuerzo y dedicación en lo que realmente estás preparado, no en lo que no te corresponde.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\nA pesar de su brevedad y sencillez, o tal vez gracias a estos mismo rasgos, pocos relatos han gozado de tal popularidad a lo largo de la historia de Occidente como este conjunto de fábulas atribuídas a Esopo (620-560 a.C.). Aunque diversos autores reescribieron el corpus esópico durante la Antigüedad c

In [5]:
import re




def limpiar_texto(texto):
    texto = texto.lower()  # pasar todo a minúsculas
    texto = re.sub(r'\s+', ' ', texto)  # eliminar saltos de línea y espacios múltiples
    texto = re.sub(r'[“”‘’]', '', texto)  # comillas raras
    texto = re.sub(r'[^\w\s]', '', texto)  # eliminar caracteres no alfanuméricos
    texto = re.sub(r'\d+', '', texto)  # eliminar números
    texto = texto.replace('\xa0', ' ')  # eliminar caracteres extraños
    return texto.strip()

corpus_limpio = limpiar_texto(text)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Crear el tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus_limpio])

total_palabras = len(tokenizer.word_index) + 1
print(f"Total de palabras únicas: {total_palabras}")

# Crear secuencias de entrenamiento
input_sequences = []
palabras = corpus_limpio.split()

for i in range(1, len(palabras)):
    secuencia = palabras[:i+1]
    codificada = tokenizer.texts_to_sequences([' '.join(secuencia)])[0]
    input_sequences.append(codificada)

# Pad sequences
secuencias_padded = pad_sequences(input_sequences, padding='pre')

# Separar en X e y
X = secuencias_padded[:, :-1]
y = secuencias_padded[:, -1]

# One-hot encode y
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_palabras)


Total de palabras únicas: 7856


#### 1.1. n-Gram trial run:

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [7]:
n = 3                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min, n_max), analyzer=n_gram_type)

In [8]:
n_grams = vectorizer.fit([corpus_limpio]).get_feature_names_out()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform([corpus_limpio]).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [9]:
list(zip(n_grams,n_gram_cts))                                    # Make a list of tuples and show.

[('abajo el hombre', np.int64(1)),
 ('abajo la cueva', np.int64(1)),
 ('abajo las campanas', np.int64(1)),
 ('abajo por eso', np.int64(1)),
 ('abalanzó pero sólo', np.int64(1)),
 ('abalanzó sobre él', np.int64(1)),
 ('abandonaban los otros', np.int64(1)),
 ('abandonado en la', np.int64(1)),
 ('abandonado ese cráneo', np.int64(1)),
 ('abandonan la buena', np.int64(1)),
 ('abandonar las que', np.int64(1)),
 ('abandonar su compromiso', np.int64(1)),
 ('abandonar una morada', np.int64(1)),
 ('abandonaran cuando ellos', np.int64(1)),
 ('abandonarle después de', np.int64(1)),
 ('abandonarlo un cazador', np.int64(2)),
 ('abandonaron el lugar', np.int64(1)),
 ('abandonaron para buscar', np.int64(1)),
 ('abandonarse al abismo', np.int64(1)),
 ('abandonas la ciudad', np.int64(1)),
 ('abandones el pequeño', np.int64(1)),
 ('abandonó su víctima', np.int64(1)),
 ('abarcar más de', np.int64(1)),
 ('abatida él le', np.int64(1)),
 ('abatido en una', np.int64(1)),
 ('abatiera su asilo', np.int64(1)),
 

#### 1.2. Train by making a dictionary based on n-Grams:

In [10]:
n = 3                                                           # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [11]:
nltk.download('punkt_tab')

n_grams = vectorizer.fit([corpus_limpio]).get_feature_names_out()           # A list of n-Grams.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])  # El (n-1)-grama
    next_word = words[-1]               # Palabra siguiente
    if a_nm1_gram not in my_dict:
        my_dict[a_nm1_gram] = [next_word]
    else:
        my_dict[a_nm1_gram].append(next_word)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
# Ver parte del diccionario
for k in list(my_dict)[:10]:
    print(f"{k} ➜ {my_dict[k]}")

abajo el ➜ ['hombre']
abajo la ➜ ['cueva']
abajo las ➜ ['campanas']
abajo por ➜ ['eso']
abalanzó pero ➜ ['sólo']
abalanzó sobre ➜ ['él']
abandonaban los ➜ ['otros']
abandonado en ➜ ['la']
abandonado ese ➜ ['cráneo']
abandonan la ➜ ['buena']


In [13]:
# View the dictionary.
my_dict

{'abajo el': ['hombre'],
 'abajo la': ['cueva'],
 'abajo las': ['campanas'],
 'abajo por': ['eso'],
 'abalanzó pero': ['sólo'],
 'abalanzó sobre': ['él'],
 'abandonaban los': ['otros'],
 'abandonado en': ['la'],
 'abandonado ese': ['cráneo'],
 'abandonan la': ['buena'],
 'abandonar las': ['que'],
 'abandonar su': ['compromiso'],
 'abandonar una': ['morada'],
 'abandonaran cuando': ['ellos'],
 'abandonarle después': ['de'],
 'abandonarlo un': ['cazador'],
 'abandonaron el': ['lugar'],
 'abandonaron para': ['buscar'],
 'abandonarse al': ['abismo'],
 'abandonas la': ['ciudad'],
 'abandones el': ['pequeño'],
 'abandonó su': ['víctima'],
 'abarcar más': ['de'],
 'abatida él': ['le'],
 'abatido en': ['una'],
 'abatiera su': ['asilo'],
 'abatieron con': ['sus'],
 'abatirlo descargó': ['contra'],
 'abatió su': ['hacha'],
 'abejas causa': ['de'],
 'abejas envidiosas': ['las'],
 'abejas probó': ['gustó'],
 'abejas volviendo': ['de'],
 'abeto diciendo': ['soy'],
 'abeto el': ['espino'],
 'abierta

#### 1.3. Predict the next word:

In [14]:
input_str = 'el cuervo'
print(input_str in my_dict)


True


In [15]:
from random import randint, seed

In [17]:
# Helper function that picks the following word.
def predict_next(a_nm1_gram):
    if a_nm1_gram not in my_dict:
        return '[desconocido]'  # opcional: puedes retornar None o un mensaje
    value_list_size = len(my_dict[a_nm1_gram])
    i_pick = randint(0, value_list_size - 1)  # importante: -1 para que esté dentro del rango
    return my_dict[a_nm1_gram][i_pick]                # Return the randomly chosen next word.

In [19]:
# Test 1: Predecir una palabra aleatoria
input_str = 'el cuervo'  # asegúrate de usar un (n-1)-grama válido
print(predict_next(input_str))

discutían


In [20]:
# Another test.
# Ver repeticiones para estimar probabilidad
input_str = 'el cuervo'
for i in range(10):
    print(predict_next(input_str))

para
la
contestó
gritón
hermes
una
discutían
viéndose
sus
viéndose


#### 1.4. Predict a sequence:

In [21]:
# Initialize the random seed.
seed(123)

In [22]:
my_seed_str = 'el lobo'
print(my_seed_str in my_dict)

True


In [23]:
a_nm1_gram = my_seed_str
output_string = my_seed_str                                         # Initialize the output string.
while a_nm1_gram in my_dict:
    next_word = predict_next(a_nm1_gram)
    output_string += " " + next_word
    words = nltk.word_tokenize(output_string)
    a_nm1_gram = ' '.join(words[-n+1:])  # actualiza el (n-1)-grama con las últimas n-1 palabras                           # Update a_nm1_gram.

In [24]:
# Output the predicted sequence.

print("Texto generado:")
print(output_string)

Texto generado:
el lobo burlado insistió el año pasado injuriaste mis padres pero en cambio incapaz hasta de probarlo encontró una compensación apropiada la manera de su diario lavado en el bolsillo del otro al volverse el carnicero notar la falta de los viajeros retrocedieron espantados excepto uno de sus vides los harían producir uvas más finas las avispas dijeron que estaba loco pero después ante los poderosos que ser humillados por los aires te daré dijo todas las inclinaciones pero olvidó dotarle del pudor no se aparenta lo que pregonas con tus dientes eso también lo hace con intenciones de maldad la víctima despertó la liebre corrió en el establo que pareciera que tiene cien ojos hasta tanto no necesitaría más del doble de peso tratar de emprender una acción que creas te beneficia ve primero que hicieron fue matar los perros no puedo hacerle mucho daño el hombre reconociendo el presente con sinceridad que sentir dolor por la espalda todo el mundo con dos bolsos cada hombre según 